In [3]:
company_bd = "사옥", "정부청사", "사무소", "대형마트", "상사"
industry_bd = "금속", "공장", "산업", "주차장", "처리시설", "발전소", "모터스", "공업", "테크"
individual_bd = "단독주택"
community_bd = "아파트", "공동주택", "빌라"

In [4]:
import pandas as pd
import os
from glob import glob

CSV_PATH = r"C:\Users\Minseok\Desktop\classfication\data\axis\csv"
os.chdir(CSV_PATH)
csv_list = glob("*.csv")

pt_list = []
rad = 10

for csv in csv_list:
    df_csv = pd.read_csv(csv, index_col=None)
    print("* " + csv + " Loaded.")
    
    for id in df_csv["OID"]:
        if ((int(id) % (2*rad)) == rad) and ((int(id) % (rad*600)) < 300):
            lat = df_csv["lat_e"][int(id)-1]
            lon = df_csv["lon_e"][int(id)-1] 
            pt_list.append([id, lat, lon])

        if ((int(id) % (2*rad)) == 0) and ((int(id) % (rad*300)) < 300) and ((int(id) / (rad*300)) >= 1):
            lat = df_csv["lat_e"][int(id)-1] 
            lon = df_csv["lon_e"][int(id)-1] 
            pt_list.append([id, lat, lon])

print("*** Points selected: " + str(len(pt_list)))

* bottom150x300.csv Loaded.
* down150x300.csv Loaded.
* mid150x300.csv Loaded.
* up150x300.csv Loaded.
*** Points selected: 1324


In [ ]:
# 지도에서 좌표를 통한 검색기 #
import requests
import json

def append_col(bd_list, col):
    try:
        bd_list.append(col)
    except:
        bd_list.append("none")
        
attributes = ['bd_st', 'bd_jibun', 'bd_lat', 'bd_lon']
total_bd = pd.DataFrame(columns = attributes)
count = 0

########################################################
CLIENT_ID = "SDFSDF"
CLIENT_SECRET = "SDFJDFJ"

for point in pt_list:    
    
    headers = {'X-NCP-APIGW-API-KEY-ID': CLIENT_ID,
        'X-NCP-APIGW-API-KEY': CLIENT_SDCRET,
        'Content-Type': 'application/x-www-form-urlencoded'}
    center_cord = str(point[1]) + "," + str(point[2])
    geo_url = 'https://naveropenapi.apigw.ntruss.com/map-geocode/v2/geocode?'    
    data = 'query=' + location + "&coordinate=" + center_cord

    response = requests.get(geo_url, headers=headers, data=data)
    results = json.loads(response.content)
    buildings.extend(results['results'])
    time.sleep(1)

    # Give ping to handle request error...
    while "next_page_token" in results:
        params['pagetoken'] = results['next_page_token'],
        response = requests.get(url, params = params)
        results = json.loads(response.content)
        buildings.extend(results['results'])
        time.sleep(1)

    print("* Number %d Detected : %d" % (count, len(buildings))) 
    
    if len(buildings) != 0:
        # Data to Collect
        bd_st =[]
        bd_jibun = []
        bd_lat = []
        bd_lon = []

        # Loading on buildings...
        for i in range(len(buildings)):
            bd = buildings[i]

            # 불발이 뜰수 있어서 키에러를 잡아줘야 한다...
            if bd['addresses'].get('roadAddress') != None:
                append_col(bd_st, bd['addresses']['roadAddress'])
            else:
                append_col(bd_st, "None")

            if bd['addresses'].get('jibunAddress') != None:
                append_col(bd_jibun, bd['addresses']['jibunAddress'])
            else:
                append_col(bd_jibun, "None")

            append_col(bd_lon, bd['addresses']['x'])
            append_col(bd_lat, bd['addresses']['y'])

        df_dict = {'bd_st':bd_st, 'bd_jibun':bd_jibun, 'bd_lat':bd_lat, 'bd_lon':bd_lon}
        bd_df = pd.DataFrame(df_dict)
        bd_df['duplicator'] = bd_df['bd_st'] + "_" + bd_df['bd_jibun']    
        bd_df.drop_duplicates(['duplicator'], inplace = True)
        total_bd = total_bd.append(bd_df)    

In [ ]:
total_bd.to_csv("산업_korea.csv",index=False)

In [ ]:
# 이미지 다운로드.
import requests

def download_image(CLIEND_ID, CLIENT_SECRET, bd_cord, Zoom, Height, Width):    
    # parameter for request
    headers = {'X-NCP-APIGW-API-KEY-ID': CLIENT_ID, 'X-NCP-APIGW-API-KEY': CLIENT_SECRET}
    width = Width
    height = Height
    lat = bd_cord[0]
    long = bd_cord[1]
    zoom = Zoom

    map_url = 'https://naveropenapi.apigw.ntruss.com/map-static/v2/raster?'
    url = map_url + "w=%d&h=%d&center=%d,%d&level=%d" % (width, height, lat, long)
    response = requests.get(url, headers=headers)

    try:
        req_img = requests.get(url)
    except IOError:
        print("ERROR: Could not generate the image - use another key or change the location")
    else:
        bd_image = open(image_name, 'wb')
        bd_image.write(req_img.content)
        bd_image.close()
        print(str(count_bd) + "--- The map has successfully been created " + str(Zoom) + "_" + str(Height))

    image_name = bd_type + "_" + str(count_bd) + "_z_" + str(Zoom) + "p_" + str(Height) +'.png'
##########################################################################
df_bd = pd.read_csv(os.path.join(DATA_PATH, csv_name))
csv_name = "산업.csv"
bd_type = "산업"

# Initial Variables
count_bd = 0  
os.chdir(r"C:\Users\Minseok\Desktop\classfication\sat_image\산업")

for bd_st, bd_jibun, bd_lat, bd_lon in zip(df_bd['bd_st'], df_bd['bd_jibun'],
                                           df_bd['bd_lat'], df_bd['bd_lon'],
                                           df_bd['duplicator']):

    map_url = 'https://naveropenapi.apigw.ntruss.com/map-static/v2/raster?'
    width = 300
    height = 300
    zoom = 18
    lat = bd_lat
    long = bd_lon

    url = map_url + "w=%d&h=%d&center=%d,%d&level=%d" % (width, height, lat, long, zoom)
    response = requests.get(url, headers=headers)

    try:
        req_img = requests.get(url)
    except IOError:
        print("ERROR: Could not generate the image - use another key or change the location")
    else:
        bd_image = open(image_name, 'wb')
        bd_image.write(req_img.content)
        bd_image.close()
        print(str(count_bd) + "--- The map has successfully been created " + str(Zoom) + "_" + str(Height))

In [7]:
# coding: UTF-8
import requests
import json
import pandas as pd
import os

buildings = []
params = {}

# Sample로 돌려보기. 
CLIENT_ID = "g5pfy8knc4"
CLIENT_SECRET = "whfWF3klMZTGfvQUhMPqPG2x5qKTMHnqrvZ5p3LF"

headers = {'X-NCP-APIGW-API-KEY-ID': CLIENT_ID,
    'X-NCP-APIGW-API-KEY': CLIENT_SECRET}

geo_url = 'https://naveropenapi.apigw.ntruss.com/map-geocode/v2/geocode'
location = '산업'
coord = '127.1054328,37.3595963'

url = geo_url + '?query=' + location + "&coord=" + coord

response = requests.get(url, headers=headers)
results = json.loads(response.content)
print(results)

{'status': 'OK', 'meta': {'totalCount': 0, 'count': 0}, 'addresses': [], 'errorMessage': ''}
